# Install dependencies

In [1]:
!pip install --quiet transformers

     |████████████████████████████████| 3.4 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 45.3 MB/s 
     |████████████████████████████████| 3.3 MB 47.6 MB/s 
     |████████████████████████████████| 895 kB 64.5 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 


# Import

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch

# Init

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print ("Device:", device)
torch.set_grad_enabled(False)

Device: cuda


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6").to(device)
model = model.to(device)

In [ ]:
# device = 0 for cuda
summarizer = pipeline("summarization", device=0)

# Logic

In [ ]:
def get_summary(input, tokenizer, model):
  txt = input['text']
  minl = input.get('min_length', 75)
  maxl = input.get('max_length', 150)
  print('minl:', minl)
  print('maxl:', maxl)

  inputs = tokenizer([txt], max_length=1024, truncation=True, return_tensors='pt').to(device)
  print('inputs:', inputs)
  output_ids = model.generate(inputs['input_ids'], num_beams=3, num_return_sequences=1, no_repeat_ngram_size=2, min_length=minl, max_length=maxl, early_stopping=True)
  print('output_ids:', output_ids)
  dec = [tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=True) for ids in output_ids]
  print('dec:', dec)
  output = dec[0].strip()
  return {'summary': output}

# Run

In [6]:
text = """
Elon Musk has shown again he can influence the digital currency market with just his tweets. After saying that his electric vehicle-making company Tesla will not accept payments in Bitcoin because of environmental concerns, he tweeted that he was working with developers of Dogecoin to improve system transaction efficiency. Following the two distinct statements from him, the world's largest cryptocurrency hit a two-month low, while Dogecoin rallied by about 20 percent. The SpaceX CEO has in recent months often tweeted in support of Dogecoin, but rarely for Bitcoin.  In a recent tweet, Musk put out a statement from Tesla that it was concerned about the rapidly increasing use of fossil fuels for Bitcoin (price in India) mining and transaction, and hence was suspending vehicle purchases using the cryptocurrency.  A day later he again tweeted saying, To be clear, I strongly believe in crypto, but it can't drive a massive increase in fossil fuel use, especially coal. It triggered a downward spiral for Bitcoin value but the cryptocurrency has stabilised since.   A number of Twitter users welcomed Musk's statement. One of them said it's time people started realising that Dogecoin is here to stay and another referred to Musk's previous assertion that crypto could become the world's future currency.
"""

In [ ]:
input = {'text': text, 'min_length': 30, 'max_length': 60}
output = get_summary(input, tokenizer, model)
print('output:', output)

In [12]:
outputs = summarizer(text, min_length=30, max_length=60)
result = ''
if len(outputs)>0:
  result = outputs[0].get('summary_text','')

print('summary:', result)

summary:  Elon Musk has shown again he can influence the digital currency market with just his tweets . The SpaceX CEO has in recent months often tweeted in support of Dogecoin, but rarely for Bitcoin . In a recent tweet, Musk put out a statement from Tesla that it was concerned about the rapidly
